# Contract Generation

When it comes to using LLMs to draft contracts, it's important to consider that contracts are often lengthy documents, sometimes spanning over 100 pages. This renders conversation-based applications impractical, as LLMs cannot generate such extensive documents, and users prefer not to interact directly with the LLM and manage its outputs themselves.

Here's a potential approach for an application to draft contracts: Rather than expecting the LLM to generate the entire document at once, we can break it down into manageable sections. Users could provide bullet points for each section, allowing the LLM to formulate the corresponding paragraphs and suggest a title for each section. Additionally, in practical scenarios, it's likely necessary to grant the LLM access to specific laws or legal references in some manner.

In [1]:
from pydantic import BaseModel, Field
from language_models.models.llm import OpenAILanguageModel
from language_models.agent import (
    Agent,
    Workflow,
    WorkflowLLMStep,
    OutputType,
    PromptingStrategy,
)
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [2]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

Here, we configure an LLM to draft detailed sections of defense contracts, ensuring they meet federal regulations and Department of Defense goals. We provide specific instructions and context to guide the AI in generating these drafts.

In [3]:
system_prompt = """You are an official at the U.S. Department of Defense.

Your primary task is to draft very detailed sections of defense contracts, ensuring they align with federal regulations and the strategic goals of the Department of Defense.

Only write about the section name you will receive and do not write any information outside of that."""

prompt = """Overall goal: {goal}

Section name: {section}
Consider this context:
{context}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=4000,
    temperature=0.2,
)

draft_contract_section = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt=prompt,
    prompt_variables=["goal", "section", "context"],
    output_type=OutputType.STRING,
    prompting_strategy=PromptingStrategy.SINGLE_COMPLETION,
    verbose=True,
)

Then, we use another LLM to refine contract sections, ensuring compliance with regulations and including relevant references. We provide the draft and context to guide the AI in enhancing the contract section.

In [4]:
system_prompt = """You are tasked with refining contract sections.

Where regulations are applied, include the references."""

prompt = """Overall goal: {goal}

Section name: {section}
Draft:
{draft}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4-32k',
    max_tokens=4000,
    temperature=0.2,
)

refine_contract_section = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt=prompt,
    prompt_variables=["goal", "section", "draft"],
    output_type=OutputType.STRING,
    prompting_strategy=PromptingStrategy.SINGLE_COMPLETION,
    verbose=True,
)

Next, we create a workflow to generate and refine contract sections using LLMs. We provide the goal and context as inputs, then process these through steps for drafting and refining the section.

In [5]:
class Inputs(BaseModel):
    goal: str = Field(description="The contract the user wants to draft")
    context: str = Field(description="The context provided for a section as bullet points")

workflow = Workflow(
    name="Generates a contract section",
    description="Uses LLMs to generate a contract section given some context",
    inputs=Inputs,
    output="refine",
    steps=[
        WorkflowLLMStep(name="draft", agent=draft_contract_section),
        WorkflowLLMStep(name="refine", agent=refine_contract_section),
    ]
)

The following shows the different sections that we want to include.

In [6]:
goal = """Draft a contract for the development, procurement, and maintenance of weapons systems
namely unmanned aerial vehicles between Gilbert Aerospace Industries Inc (referred to as employee)
and the Department of Defense (DoD) referred to as employer. Provide information about the background,
Applicable Documents, Requirements, Security"""

class ContractSection(BaseModel):
    title: str
    context: str

scope_of_work = ContractSection(
    title="Scope of Work",
    context="""The Contractor shall provide all personnel, equipment, materials, and facilities necessary to design, develop, manufacture, test, deliver, and maintain the UAV systems.

The scope includes:
- Design & Development:
- Procurement & Manufacturing:
- Maintenance & Support:
- Training & Documentation:""",
)

requirements = ContractSection(
    title="Requirements",
    context="""The following documents are incorporated by reference and made a part of this contract. All work shall conform to the specifications, standards, and guidelines contained in these documents:
- DoD Standards & Specifications:
- Federal Acquisition Regulation (FAR):
- DoD Instruction (DoDI) [Number]:
- System Requirements Document (SRD):
- Test and Evaluation Master Plan (TEMP):
- Security Classification Guide (SCG):""",
)

technical_requirements = ContractSection(
    title="Technical Requirements",
    context="""The technical requirements should include:
- Design Specifications:
- Software Development:
- System Integration:""",
)

delivery_requirements = ContractSection(
    title="Delivery Requirements",
    context="""The delivery requirements should include:
- Production Schedule:
- Quality Assurance:""",
)

maintenance_requirements = ContractSection(
    title="Maintenance Requirements",
    context="""The maintenance requirements should include:
- Maintenance Services: T
- Spare Parts: T""",
)

security_requirements = ContractSection(
    title="Security Requirements",
    context="""Compliance with cybersecurity, program protection plan etc…

The security requirements should include:
- Security Classification:
- Facility Security:""",
)

cybersecurity_requirements = ContractSection(
    title="Cybersecurity Requirements",
    context="""Compliance with cybersecurity, program protection plan etc…

The cybersecurity requirements should include:
- Incident Reporting:""",
)

period_of_performance = ContractSection(
    title="Period of Performance",
    context="""This section should include:
- Start and End Dates: Clearly specify the contract start date (e.g., "Commencement of work on [Start Date]") and the expected completion date (e.g., "Completion of all contractual obligations by [End Date]"), ensuring alignment with project milestones and deliverables.
- Milestone Deadlines: Outline critical milestone dates within the period of performance, including deadlines for design reviews, prototype delivery, testing phases, and final system delivery, to ensure progress is tracked and managed effectively.
- Options for Extension: Include provisions for possible extensions or modifications of the period of performance, detailing the Nongovernmental Organization (NGO) process and conditions under which the contract duration may be extended (e.g., unforeseen delays, additional work requirements).""",
)

payment_and_funding = ContractSection(
    title="Payment and Funding",
    context="""This section should include:
- Contract Value:
- Payment Schedule:
- Funding Obligations:""")

contract_administration = ContractSection(
    title="Contract Administration",
    context="""This section should include:
- Contracting Officer Responsibilities: Identify the designated Contracting Officer (CO) from the DoD, outlining their authority to administer the contract, make decisions, issue modifications, and handle any formal communications between the DoD and the Contractor.
- Contractor Point of Contact: Specify the primary point of contact (POC) from Gilbert Aerospace Industries Inc., responsible for managing contract performance, responding to CO inquiries, and ensuring timely submission of required deliverables and reports.
- Record-Keeping and Reporting Requirements: Detail the procedures for maintaining and submitting records related to contract performance, including regular status reports, financial documentation, and compliance records, ensuring they meet the standards set by the FAR and other applicable regulations."""
)

disputes = ContractSection(
    title="Disputes",
    context="""This section should include:
- Dispute Resolution Process: Outline the step-by-step process for resolving disputes, including initial negotiation efforts, escalation procedures, and potential use of alternative dispute resolution (ADR) methods like mediation or arbitration before resorting to litigation.
- Governing Law and Jurisdiction: Specify the applicable law governing the contract (e.g., federal law, including the Federal Acquisition Regulation) and the jurisdiction or venue where disputes will be adjudicated (e.g., U.S. Court of Federal Claims).
- Timelines for Filing Claims: Define the timeframes within which disputes must be raised and resolved, including deadlines for filing formal claims, submitting responses, and completing the dispute resolution process to prevent delays in contract performance."""
)

termination = ContractSection(
    title="Termination",
    context="""This section should include:
- Termination for Convenience: Specify the conditions under which either party may terminate the contract for convenience, including the required notice period (e.g., 30 days) and any obligations for settling outstanding payments and deliverables upon termination.
- Termination for Cause: Outline the circumstances that allow termination for cause, such as failure to perform contractual obligations, non-compliance with terms, or breach of contract. Include the procedures for notifying the offending party, providing opportunities to cure breaches, and the process for formal termination.
- Settlement of Accounts: Detail the procedures for the settlement of accounts upon termination, including the return of any government-furnished property, final payments for work performed up to the termination date, and any adjustments for partially completed work or costs incurred."""
)

signatures = ContractSection(
    title="Signatures",
    context="""This section should include:
- Authorized Signatures: Ensure that the contract is signed by authorized representatives from both the Department of Defense and Gilbert Aerospace Industries Inc., including the names, titles, and dates of signing, along with any required witness signatures."""
)

contract_sections = [
    scope_of_work,
    requirements,
    technical_requirements,
    delivery_requirements,
    maintenance_requirements,
    security_requirements,
    cybersecurity_requirements,
    period_of_performance,
    payment_and_funding,
    contract_administration,
    disputes,
    termination,
    signatures,
]

Now, we can generate a contract based on the information we provided.

In [7]:
class Step(BaseModel):
    title: str
    steps: list

class Contract(BaseModel):
    content: str
    steps: list[Step]

def generate_contract(goal: str, contract_sections: list[ContractSection]) -> Contract:
    steps = []
    content = []
    for contract_section in contract_sections:
        output = workflow.invoke({
            "goal": goal,

            "section": contract_section.title,
            "context": contract_section.context,
        })
        content.append(output.output)
        steps.append(Step(title=contract_section.title, steps=output.steps))
    return Contract(content="\n\n".join(content), steps=steps)

In [8]:
contract = generate_contract(goal, contract_sections)

Use LLM: draft
Inputs: {'goal': 'Draft a contract for the development, procurement, and maintenance of weapons systems\nnamely unmanned aerial vehicles between Gilbert Aerospace Industries Inc (referred to as employee)\nand the Department of Defense (DoD) referred to as employer. Provide information about the background,\nApplicable Documents, Requirements, Security', 'section': 'Scope of Work', 'context': 'The Contractor shall provide all personnel, equipment, materials, and facilities necessary to design, develop, manufacture, test, deliver, and maintain the UAV systems.\n\nThe scope includes:\n- Design & Development:\n- Procurement & Manufacturing:\n- Maintenance & Support:\n- Training & Documentation:'}
Prompt: Overall goal: Draft a contract for the development, procurement, and maintenance of weapons systems
namely unmanned aerial vehicles between Gilbert Aerospace Industries Inc (referred to as employee)
and the Department of Defense (DoD) referred to as employer. Provide informa

In [9]:
print(contract.content)

Section: Scope of Work

The Contractor, Gilbert Aerospace Industries Inc, hereinafter referred to as the "Contractor," shall provide all personnel, equipment, materials, and facilities necessary to perform the following tasks:

1. Design & Development: The Contractor shall design and develop advanced unmanned aerial vehicles (UAVs) as per the specifications and requirements provided by the Department of Defense (DoD), in accordance with the Defense Federal Acquisition Regulation Supplement (DFARS) 252.225-7040. This includes, but is not limited to, the conceptualization, design, development, and prototyping of UAV systems. The Contractor shall also be responsible for conducting comprehensive system and performance tests to ensure the UAVs meet the DoD's operational requirements.

2. Procurement & Manufacturing: The Contractor shall procure all necessary materials and components for the manufacturing of the UAV systems, in compliance with the Federal Acquisition Regulation (FAR) Part 25

To get a better sense of what is happening behind the scenes, we simply render the steps of the LLM-powered workflow.

In [14]:
def render_reasoning(steps):
    for i, step in enumerate(steps):
        names = {
            "system_prompt": "System Prompt",
            "prompting_strategy": "Prompting Strategy",
            "prompt": "Prompt",
            "raw_output": "Raw Output",
            "observation": "Observation",
            "final_answer": "Final Answer",
            "tool_use": "Tool Use",
            "tool_output": "Tool Output",
        }
        string = "Use LLM"
        print(string)
        print("=" * len(string))
        print()
        for entry in step.steps:
            print(names[entry.name])
            print("-" * len(entry.name))
            if entry.name == "tool_use":
                print(f"Thought: {entry.content.thought}")
                print()
                print(f"Used: {entry.content.used}")
                print()
                print(f"Arguments: {entry.content.arguments}")
            elif entry.name == 'final_answer':
                if entry.content.thought is not None:
                    print(f"Thought: {entry.content.thought}")
                    print()
                print(f"Output: {entry.content.output}")
            else:
                print(entry.content)

            if entry.name != "final_answer" or i != len(steps) - 1:
                print()

In [19]:
for step in contract.steps:
    string = "Contract Section"
    print(string)
    print("-" * len(string))
    print(step.title)
    print()
    render_reasoning(step.steps)
    print()

Contract Section
----------------
Scope of Work

Use LLM

System Prompt
-------------
You are an official at the U.S. Department of Defense.

Your primary task is to draft sections of defense contracts, ensuring they align with federal regulations and the strategic goals of the Department of Defense.

Only write about the section name you will receive and do not write any information outside of that.

Prompting Strategy
------------------
single_completion

Prompt
------
Overall goal: Draft a contract for the development, procurement, and maintenance of weapons systems
namely unmanned aerial vehicles between Gilbert Aerospace Industries Inc (referred to as employee)
and the Department of Defense (DoD) referred to as employer. Provide information about the background,
Applicable Documents, Requirements, Security

Section name: Scope of Work
Consider this context:
The Contractor shall provide all personnel, equipment, materials, and facilities necessary to design, develop, manufacture, t